In [ ]:
from tkinter import *
import time
import datetime
import pyttsx3
import speech_recognition as sr
from threading import Thread
import requests
from bs4 import BeautifulSoup


def shut_down():
    p1=Thread(target=speak,args=("Shutting down. Thankyou For Using Our Sevice. Take Care, Good Bye.",))
    p1.start()
    p2 = Thread(target=transition2)
    p2.start()
    time.sleep(7)
    root.destroy()

def transition2():
    global img1
    global flag
    global flag2
    global frames
    global canvas
    local_flag = False
    for k in range(0,5000):
        for frame in frames:
            if flag == False:
                canvas.create_image(0, 0, image=img1, anchor=NW)
                canvas.update()
                flag = True
                return
            else:
                canvas.create_image(0, 0, image=frame, anchor=NW)
                canvas.update()
                time.sleep(0.1)

def web_scraping(qs):
    global flag2
    global loading
    try:
        link = 'https://www.allrecipes.com/search?q='
        query = qs.replace(" ", "-")
        find_recipe = requests.get(link+query)
        soup = BeautifulSoup(find_recipe.content, 'html.parser')
        s = soup.find('div', class_='comp search-results__content mntl-block')
        content = s.find_all('a')
        recipe_link = content[0].get('href')
        fetch_recipe = requests.get(recipe_link)
        soup = BeautifulSoup(fetch_recipe.content, 'html.parser')

        div0 = soup.find_all('div',class_="comp recipe-details mntl-recipe-details")
        div1 = soup.find_all("div", class_='comp mntl-structured-ingredients')
        div2 = soup.find_all('ol', class_="comp mntl-sc-block-group--OL mntl-sc-block mntl-sc-block-startgroup")
        div3  = soup.find_all('table', class_="mntl-nutrition-facts-summary__table")

        if len(div0)!=0:
            answer = div0[0].text
        elif len(div1) != 0:
            answer = div1[0].text+"\n"+div1[0].find_next_sibling("div").text
        elif len(div2) != 0:
            answer = div2[0].find_next("span").text+"\n"+div2[0].find_next("div",class_="kCrYT").text
        elif len(div3)!=0:
            answer = div3[1].text
        elif flag==True:
            wiki = 'https://en.wikipedia.org/wiki/'+qs
            page2 = requests.get(wiki)
            soup = BeautifulSoup(page2.text, 'html.parser')
            title = soup.select("#firstHeading")[0].text

            paragraphs = soup.select("p")
            for para in paragraphs:
                if bool(para.text.strip()):
                    answer = title + "\n" + para.text
                    break
    except:
        answer = "Sorry. I could not find the desired results"


    canvas2.create_text(10, 225, anchor=NW, text=answer, font=('Candara Light', -25,'bold italic'),fill="white", width=350)
    flag2 = False
    loading.destroy()

    p1=Thread(target=speak,args=(answer,))
    p1.start()
    p2 = Thread(target=transition2)
    p2.start()


def speak(text):
    global flag
    engine.say(text)
    engine.runAndWait()
    flag=False


def wishme():
    hour = datetime.datetime.now().hour

    if 0 <= hour < 12:
        text = "Good Morning sir. I am Foodie. Which dish do you wanna cook today? (Click on ask me!)"
    elif 12 <= hour < 18:
        text = "Good Afternoon sir. I am Foodie. Which dish do you wanna cook today?(Click on ask me!)"
    else:
        text = "Good Evening sir. I am Foodie. Which dish do you wanna cook today?(Click on ask me!)"

    canvas2.create_text(10,10,anchor =NW , text=text,font=('Candara Light', -25,'bold italic'), fill="white",width=350)
    p1=Thread(target=speak,args=(text,))
    p1.start()
    p2 = Thread(target=transition2)
    p2.start()


def takecommand():
    global loading
    global flag
    global flag2
    global canvas2
    global query
    global img4
    if flag2 == False:
        canvas2.delete("all")
        canvas2.create_image(0,0, image=img4, anchor="nw")

    speak("I am listening.")
    flag= True
    r = sr.Recognizer()
    r.dynamic_energy_threshold = False
    r.energy_threshold = 4000
    with sr.Microphone() as source:
        print("Listening...")
        #r.pause_threshold = 3
        audio = r.listen(source,phrase_time_limit=4)

    try:
        print("Recognizing..")
        query = r.recognize_google(audio, language='en-in')
        print(f"user Said :{query}\n")
        query = query.lower()
        canvas2.create_text(490, 120, anchor=NE, justify = RIGHT ,text=query, font=('fixedsys', -30),fill="white", width=350)
        global img3
        loading = Label(root, image=img3, bd=0)
        loading.place(x=900, y=622)

    except Exception as e:
        print(e)
        speak("Say that again please")
        return "None"




def main_window():
    global query
    wishme()
    while True:
        if query != None:
            if 'shutdown' in query or 'quit' in query or 'stop' in query or 'goodbye' in query:
                shut_down()
                break
            else:
                web_scraping(query)
                query = None

    


if __name__ == "__main__":
    loading = None
    query = None
    flag = True
    flag2 = True

    engine = pyttsx3.init() # Windows
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[1].id)
    rate = engine.getProperty('rate')
    engine.setProperty('rate', rate-10)

    root=Tk()
    root.title("Foodie Bot")
    root.geometry('1360x690+-5+0')
    root.configure(background='white')

    img1= PhotoImage(file=r'D:\Jio Institute\Q2\NLP\Assignments\UI\chatbot-image.png')
    img2= PhotoImage(file=r'D:\Jio Institute\Q2\NLP\Assignments\UI\button-green.png')
    img3= PhotoImage(file=r'D:\Jio Institute\Q2\NLP\Assignments\UI\icon.png')
    img4= PhotoImage(file=r'D:\Jio Institute\Q2\NLP\Assignments\UI\terminal.png')
    background_image=PhotoImage(file=r'D:\Jio Institute\Q2\NLP\Assignments\UI\last.png')

    f = Frame(root,width = 1360, height = 690)
    f.place(x=0,y=0)
    f.tkraise()
    front_image = PhotoImage(file=r"D:\Jio Institute\Q2\NLP\Assignments\UI\front2.png")
    okVar = IntVar()
    btnOK = Button(f, image=front_image,command=lambda: okVar.set(1))
    l = Label(root, text = "Click on the image!!")
    l.pack()
    btnOK.place(x=510,y=180)
    f.wait_variable(okVar)
    f.destroy()    

    background_label = Label(root, image=background_image)
    background_label.place(x=0, y=0)

    frames = [PhotoImage(file=r'D:\Jio Institute\Q2\NLP\Assignments\UI\chatgif.gif',format = 'gif -index %i' %(i)) for i in range(20)]
    canvas = Canvas(root, width = 500, height = 596)
    canvas.place(x=10,y=10)
    canvas.create_image(0, 0, image=img1, anchor=NW)
    question_button = Button(root,image=img2, bd=0, command=takecommand)
    question_button.place(x=200,y=525)

    frame=Frame(root,width=500,height=596)
    frame.place(x=625,y=10)
    canvas2=Canvas(frame,bg='#FFFFFF',width=500,height=596,scrollregion=(0,0,500,900))
    vbar=Scrollbar(frame,orient=VERTICAL)
    vbar.pack(side=RIGHT,fill=Y)
    vbar.config(command=canvas2.yview)
    canvas2.config(width=500,height=596, background="black")
    canvas2.config(yscrollcommand=vbar.set)
    canvas2.pack(side=LEFT,expand=True,fill=BOTH)
    canvas2.create_image(0,0, image=img4, anchor="nw")

    task = Thread(target=main_window)
    task.start()
    root.mainloop()

Listening...
Recognizing..
user Said :Dhoni birthday



Exception in thread Thread-28:
Traceback (most recent call last):
  File "C:\Users\Hany\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Hany\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Hany\AppData\Local\Temp\ipykernel_20808\1304685194.py", line 93, in speak
  File "C:\Users\Hany\anaconda3\lib\site-packages\pyttsx3\engine.py", line 177, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


Listening...
Recognizing..

Listening...
Recognizing..
user Said :take



Exception in thread Thread-34:
Traceback (most recent call last):
  File "C:\Users\Hany\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Hany\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Hany\AppData\Local\Temp\ipykernel_20808\1304685194.py", line 93, in speak
  File "C:\Users\Hany\anaconda3\lib\site-packages\pyttsx3\engine.py", line 177, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


Listening...
Recognizing..

Listening...
Recognizing..
user Said :chicken roast



Exception in thread Thread-39:
Traceback (most recent call last):
  File "C:\Users\Hany\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Hany\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Hany\AppData\Local\Temp\ipykernel_20808\1304685194.py", line 93, in speak
  File "C:\Users\Hany\anaconda3\lib\site-packages\pyttsx3\engine.py", line 177, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
